### First steps with scikit-learn - training a perceptron  

In [1]:
from sklearn import datasets
import numpy as np

In [2]:
iris = datasets.load_iris()
X = iris.data[:, [2,3]]
y = iris.target

In [3]:
print("Class labels:", np.unique(y))

Class labels: [0 1 2]


#### Retorna las tres etiquetas de clase que tenemos para nuestro dataset:
- Iris-setosa
- Iris-versicolor
- Iris-virginica

##### Train and test datasets
Para ver que tan bien se desempeña nuestro modelo con datos que no conoce, procedemos a separar el dataset en training y test

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3,
                                                    random_state=1,
                                                    stratify=y)

- np.bincount(y)

Cuenta el número de ocurrencias de cada valor en y, y lo entrega a modo de lista

In [7]:
print("Labels counts in y:", np.bincount(y))
print("Labels counts in train:", np.bincount(y_train))
print("Labels counts in test:", np.bincount(y_test))

Labels counts in y: [50 50 50]
Labels counts in train: [35 35 35]
Labels counts in test: [15 15 15]


Muchos algorítmos de Machine Learning requieren, además, el escalado de características para un desempeño óptimo, como vimos en el caso del descenso del gradiente.

Utilizaremos la la clase *StandardScaler* del módulo **preprocessing** de scikit-learn

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

Utilizando el método *fit*, StandardScaler estíma los parámetros $\mu$ (media) y $\sigma$ (desviación estandar) para cada característica del dataset.

Y con *transform* estandarizamos los datos de entrenamiento utilizando los parámetros, $\mu$ y $\sigma$

### Entrenemos nuestro modelo de perceptron
La mayoría de algorítmos en scikit-learn ya soportan la clasificación multiclase por defecto utilizando el método **one-vs.-rest(OvR)**.

In [10]:
from sklearn.linear_model import Perceptron

ppn = Perceptron(eta0=0.01, random_state=1)
ppn.fit(X_train_std, y_train)

Perceptron(eta0=0.01, random_state=1)

Habiendo entrenado nuestro modelo en scikit-learn, podemos hacer predicciones a través del método *predict*:

In [11]:
y_pred = ppn.predict(X_test_std)

In [12]:
print("Misclassified examples: %d" % (y_test != y_pred).sum())

Misclassified examples: 2


### Calculémos la precisión en la clasificación de nuestro perceptron sobre el conjunto de test

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
print("Accuracy: %.3f" % accuracy_score(y_test, y_pred))

Accuracy: 0.956


Cada clasificador en scikit-learn tiene su método *score* para calcular la precision del clasificador combinando la llamada *predict* con *accuracy_score*: 

In [15]:
print("Accuracy: %.3f" % ppn.score(X_test_std, y_test))

Accuracy: 0.956
